## Housing Price Prediction with Random Forest Case Study Yogyakarta

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
dataset = pd.read_csv("rumahcom_clean.csv")
dataset.head()

,No,harga,lokasi,luas_bangunan,luas_tanah,kamar,kamar_mandi,listrik,interior,parkir,sertifikat
0,1,980290930.0,Sleman,90,100,3,2,1300,Tak Berperabot,2.0,SHM - Sertifikat Hak Milik
1,2,490000000.0,Sleman,60,90,3,1,1300,Tak Berperabot,1.0,SHM - Sertifikat Hak Milik
2,3,780293930.0,Sleman,84,100,3,2,1300,Tak Berperabot,2.0,SHM - Sertifikat Hak Milik
3,4,780939000.0,Sleman,84,100,3,2,1300,Tak Berperabot,1.0,SHM - Sertifikat Hak Milik
4,5,1799000000.0,Yogyakarta,154,130,3,3,2200,Tak Berperabot,2.0,SHM - Sertifikat Hak Milik
